In [1]:
!pip install datasets

In [2]:
import torch 
from datasets import load_dataset

full_dataset=load_dataset("cnn_dailymail",version="3.0.0")

In [5]:
import torch 
from datasets import load_dataset

full_dataset=load_dataset("cnn_dailymail",version="3.0.0")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [3]:
sample_size=50
sample=(full_dataset["train"]
       .filter(lambda r: "CNN" in r["article"][:25])
       .shuffle(seed=42)
        .select(range(sample_size))
       
       )
sample

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 50
})

In [4]:
display(sample.to_pandas())

,article,highlights,id
0,(CNN) -- A magnitude 6.7 earthquake rattled Pa...,Papua New Guinea is on the so-called Ring of F...,8093dba7bc2260c26f18939826909ef27549c758
1,(CNN) -- Pakistan took big steps towards level...,Australia collapse to 88 all out on opening da...,67d626156f971d0bf55e5f2a48e1ed965eb622a6
2,(CNN) -- Federal prosecutors are pushing to fo...,Jared Loughner is refusing the government's re...,0d02fb8f0d406db956b128a5c1cc7bf3f13860a6
3,"Centennial, Colorado (CNN) -- McKayla Hicks sa...",Shooting victim McKayla Hicks went to hearing ...,39aee887c6d34bd311c826142b14037e6f2639ee
4,(CNN) -- Double-amputee sprinter Oscar Pistori...,Oscar Pistorius to become first double-amputee...,cc83ecdf08f0b598c3b97b3e2819c7e0ae7ca4f2
5,(CNN) -- A grand jury has indicted Texas Gov. ...,"NEW: Perry lawyer calls indictments ""political...",51fb6465303595cb201b427ca04b594b182a9722
6,(CNN)An Argentine prosecutor said Friday there...,Prosecutor to judge: Enough evidence for inves...,f4d3394791035a0571f1841d5d21661fdb39d74f
7,"Warsaw, Poland (CNN) -- European football's go...",NEW: UEFA president Michel Platini urges fans ...,76ba8e9110a66a1b1293abe34ef4fab254371af8
8,(CNN) -- Two issues -- security and immigratio...,A new high-level group to discuss economic coo...,fbca9bf96c440bbfab59de6bd5f6d06ed609ed99
9,(CNN) -- More than 100 police officers and ot...,Four inmates escape from jail in St. Tammany P...,a91b42eb3bfaa9dd1d6fe5e07d595f0acdbf29bc


In [5]:
example_article=sample["article"][0]
example_summary=sample["highlights"][0]
print(f"Article:\n{example_article}\n")
print(f"Summary:\n{example_summary}\n")

Article:
(CNN) -- A magnitude 6.7 earthquake rattled Papua New Guinea early Friday afternoon, according to the U.S. Geological Survey. The quake was centered about 200 miles north-northeast of Port Moresby and had a depth of 28 miles. No tsunami warning was issued, according to the Tsunami Warning Center. Papua New Guinea is on the so-called Ring of Fire, an arc of fault lines circling the Pacific Basin that is prone to frequent earthquakes and volcanic eruptions.

Summary:
Papua New Guinea is on the so-called Ring of Fire .
It's on an arc of fault lines that is prone to frequent earthquakes .
No tsunami warning was issued .



In [6]:
! pip install simplet5

In [15]:
from simplet5 import SimpleT5
model = SimpleT5()

Global seed set to 42


In [7]:
import pandas as pd
import torch
import gc
from transformers import AutoTokenizer, T5ForConditionalGeneration

In [8]:
def batch_generator(data: list, batch_size: int):
    """
    Creates batches of size `batch_size` from a list.
    """
    s = 0
    e = s + batch_size
    while s < len(data):
        yield data[s:e]
        s = e
        e = min(s + batch_size, len(data))


def summarize_with_t5(
    model_checkpoint: str, articles: list, batch_size: int = 8
) -> list:
    """
    Compute summaries using a T5 model.
    This is similar to a `pipeline` for a T5 model but does tokenization manually.

    :param model_checkpoint: Name for a model checkpoint in Hugging Face, such as "t5-small" or "t5-base"
    :param articles: List of strings, where each string represents one article.
    :return: List of strings, where each string represents one article's generated summary
    """
    if torch.cuda.is_available():
        device = "cuda:0"
    else:
        device = "cpu"

    model = T5ForConditionalGeneration.from_pretrained(
        model_checkpoint, cache_dir="../working/cache"
    ).to(device)
    tokenizer = AutoTokenizer.from_pretrained(
        model_checkpoint, model_max_length=1024, cache_dir="../working/cache"
    )

    def perform_inference(batch: list) -> list:
        inputs = tokenizer(
            batch, max_length=1024, return_tensors="pt", padding=True, truncation=True
        )

        summary_ids = model.generate(
            inputs.input_ids.to(device),
            attention_mask=inputs.attention_mask.to(device),
            num_beams=2,
            min_length=0,
            max_length=40,
        )
        return tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

    res = []

    summary_articles = list(map(lambda article: "summarize: " + article, articles))
    for batch in batch_generator(summary_articles, batch_size=batch_size):
        res += perform_inference(batch)

        torch.cuda.empty_cache()
        gc.collect()

    # clean up
    del tokenizer
    del model
    torch.cuda.empty_cache()
    gc.collect()
    return res

In [9]:
t5_small_summaries = summarize_with_t5("t5-small", sample["article"])

In [10]:
reference_summaries = sample["highlights"]

In [11]:
display(
    pd.DataFrame.from_dict(
        {
            "generated": t5_small_summaries,
            "reference": reference_summaries,
        }
    )
)

,generated,reference
0,a magnitude 6.7 earthquake rattles Papua new G...,Papua New Guinea is on the so-called Ring of F...
1,the two-Test cricket series is being played in...,Australia collapse to 88 all out on opening da...
2,federal prosecutors want jared Lee Loughner to...,Jared Loughner is refusing the government's re...
3,"new: ""he tried to kill people,"" a 17-year-old ...",Shooting victim McKayla Hicks went to hearing ...
4,double-amputee sprinter Oscar Pistorius will c...,Oscar Pistorius to become first double-amputee...
5,new: a grand jury indicted the governor on cha...,"NEW: Perry lawyer calls indictments ""political..."
6,prosecutor says there is enough evidence to co...,Prosecutor to judge: Enough evidence for inves...
7,"UEFA says it is acting over ""the setting-off a...",NEW: UEFA president Michel Platini urges fans ...
8,new: a new high-level group will discuss econo...,A new high-level group to discuss economic coo...
9,"police, others searching for murder suspect in...",Four inmates escape from jail in St. Tammany P...


In [12]:
def compute_rouge_score(generated: list, reference: list) -> dict:
    """
    Compute ROUGE scores on a batch of articles.

    This is a convenience function wrapping Hugging Face `rouge_score`,
    which expects sentences to be separated by newlines.

    :param generated: Summaries (list of strings) produced by the model
    :param reference: Ground-truth summaries (list of strings) for comparison
    """
    generated_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in generated]
    reference_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in reference]
    return rouge_score.compute(
        predictions=generated_with_newlines,
        references=reference_with_newlines,
        use_stemmer=True,
    )
# ROUGE scores for o

In [18]:
#!pip install evaluate
!pip install rouge_score

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=a6dcc464be81534a273f07c5e9189c2cdf8d1c786662db86b55ce48ce9e2d5ef
  Stored in directory: c:\users\mutahir manzoor\appdata\local\pip\cache\wheels\9b\3d\39\09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge_score


In [19]:
import evaluate
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")

rouge_score = evaluate.load("rouge")

[nltk_data] Downloading package punkt to C:\Users\Mutahir
[nltk_data]     Manzoor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
compute_rouge_score(t5_small_summaries, reference_summaries)

{'rouge1': 0.3083524228022702,
 'rouge2': 0.09746626316452917,
 'rougeL': 0.218882579864896,
 'rougeLsum': 0.28516720966053954}